## Contact
#### [linkedin](https://www.linkedin.com/in/justin-isinguzo-464bb083/)  &emsp;  [twitter](https://twitter.com/barrizm)  &emsp;[facebook](https://facebook.com/barrizmjay)

<h1><center>Linear programming(Transportation Problem)</center></h1> <br>  <h2><center>Case study: NNPC and Downstream sector</center></h2>

This iPython Notebook seeks to build an abstract Linear program in python with pyomo module.
<hr>
<p>Problem formulation: There are a certain amount of refineries and depots(3 and 22 respectively in this case) located in different parts of the country, a change of flow is needed to save cost and also boost the efficiency of supply to different parts of the country.</p>

* Objective: Minimize the Cost of Transporting refined products from the 3 refineries in Nigeria to 22 depots scattered across the country
* LP Form: Minimization(pyomo)
* Decision Variables: sixty six(66) Variables which source to destination is cheaper.
* Constraints: Demands must be met and supply cannot exceed production.

<div class="alert alert-block alert-info">
<b>Why abstract model</b><br>
Abstract model is prefered in this case because of its flexibility in terms of the large number of variables and changing values or adding variables for further research or when the need arises. All you need to do is add the new or edited input to the data file.
</div>

<div class="alert alert-block alert-warning">
<b>Note:</b>
When using abstract models, it is very important to know that it must work with a data file in `.dat`,`.csv`,`.xlsx` etc as this is where the model picks the sets and parameters to be used during iterations.
</div>

In [21]:
#Importing the module
from pyomo.environ import * 

## Data used
From the Nigerian National Petroleum Coporation quarterly report. Files containing data have been uploaded as .xlxs and .csv files in the [data folder](https://github.com/Barrizm/Transportatation-Problem/tree/main/Data)
<br>

Data content
* Refineries 
* Depots
* Cost of transportation
* Quantity needed by the depots(Demand)
* Quantity produced by the refineries (Supply)


## A quick dive into the model


This creates a "container" where all the action will be performed.
When private `refineries` becomes fully functional, the number of `refineries` can be increased as well as the `depots` in the data file and the `optimal solution` will be found.   
its sets and parameters are from an outsource(an external file) for execution.

In [11]:
model = AbstractModel()

The set that will be used to construct the needed matrix for iterations. An input in the data file.<br>

 $Sets = (Refineries, Depots)$




In [12]:
# Define sets
model.Refineries = Set()
model.Depots = Set()


These are the values of the `demand` from Depots, `supply` by the Refineries and the `cost` of moving these products from the Refineries to Depots. An input in the data file

In [13]:
## Define parameters
model.refinery = Param(model.Refineries)
model.depot = Param(model.Depots)
model.costs = Param(model.Refineries, model.Depots)

Desicion variables defined by the `Var` component from source to destination, that is from refineries to depots


In [14]:
## Define variables
model.flow = Var(model.Refineries, model.Depots, within = NonNegativeReals)

Implementing the `objective` function

$\min cost = \sum_{i \in I} \sum_{j \in J} c_{i,j} x_{i,j}$

In [15]:
## Define Objective Function
def costRule(model):
   return sum(
       model.costs[n,i] * model.flow[n,i]
       for n in model.Refineries
       for i in model.Depots
    )
model.SolverResult=Objective(rule=costRule)


<div class="alert alert-block alert-info">
    Implementing the <b>constraints</b> components along side its <b>rule</b>. 
</div>


In this case the supply must exceed demand to avoid scarcity 

$\sum_{i \in I} x_{i,j} \geq b_{j}$, <br> $\forall j \in J$

In [16]:

## Satisfy demands
def minDemandRule(model,bar):
    return sum(model.flow[i, bar] for i in model.Refineries) >= model.depot[bar]
model.depotConstraint = Constraint(model.Depots, rule=minDemandRule)



the quantity demanded should not exceed what the company has produced.

$\sum_{j \in J} x_{i,j} \leq a_{i}$, <br> $\forall i \in I$

In [ ]:
## Satisfy supplies
def maxSupplyRule(model,warehouse):
    return sum(model.flow[warehouse, j] for j in model.Depots) <= model.refinery[warehouse]
model.refineryConstraint = Constraint(model.Refineries, rule=maxSupplyRule)


Running the program in the note book with [$\color{red}{\text{ xpress_direct}}$](https://www.fico.com/) solver and making reference to the data file [$\color{red}{\text{ pms_cost.dat}}$](https://github.com/Barrizm/Transportatation-Problem/blob/main/Data/pms_cost.dat) with the parameters herein result for optimality 

In [22]:
# solver used 
opt = SolverFactory('xpress_direct')

#creating an instance wiht the data file containing the parameter
instance = model.create_instance("pms_cost.dat")
results = opt.solve(instance)

# instance.display()
instance.solutions.store_to(results)
results.write()




# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 2171099709.0
  Upper bound: 2171099709.0
  Number of objectives: 1
  Number of constraints: 25
  Number of variables: 66
  Number of integer variables: 0
  Number of continuous variables: 66
  Number of nonzeros: 132
  Sense: 1
  Number of solutions: 1
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Name: Xpress 40.1.2
  Status: ok
  Wallclock time: 0.08663702011108398
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.

# Result interpretation

An optimal result was found and projected an estimated cost of $\color{blue}{\text{ N2,171,099,709}}$  (two billion one hundred and seventy one million, ninety nine thousand, seven hundred and nine naira) to transport Premium Motor Spirit to all parts of the country which is the summation of all the values from the result above.<br>
This information:-

Example above

$\color{blue}{\text{flow[KRPC, GOMBE]:}}$
<br>
&emsp;$\color{blue}{\text{Value: 68518}}$<br>

  means that `KRPC` will satisfy the depot demands from `GOMBE`which is `68518`. This translation applies to all as same model can be used for Automated Gas Oil(AGO) and House Hold Kerosene(HHK) with the [data](https://github.com/Barrizm/Transportatation-Problem/tree/main/Data)
  
  
<div class="alert alert-block alert-info">

These results can be compared to the current cost of transporting these products hence make cost adjustments where necessary.<br>
</div>

[<b>complete thesis</b>](https://github.com/Barrizm/Transportatation-Problem/blob/main/Thesis_transportation_NNPC_Msc.pdf)